In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained model (you can use a TensorFlow model of your choice)
model = tf.saved_model.load("path/to/your/model")

# Load the label map for object detection
with open("path/to/your/label_map.txt", "r") as f:
    labels = f.read().splitlines()

# Function to perform object detection
def detect_objects(frame):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    
    # Perform detection
    detections = model(input_tensor)
    
    return detections

# Initialize video capture from the drone's camera
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break

    # Resize frame for detection
    height, width, _ = frame.shape
    resized_frame = cv2.resize(frame, (640, 640))
    
    # Perform object detection
    detections = detect_objects(resized_frame)

    # Loop through detections
    for i in range(detections['detection_boxes'].shape[1]):
        confidence = detections['detection_scores'][0][i].numpy()
        if confidence > 0.5:  # Threshold for detection
            box = detections['detection_boxes'][0][i].numpy()
            (y_min, x_min, y_max, x_max) = (box * np.array([height, width, height, width])).astype("int")
            label = int(detections['detection_classes'][0][i].numpy())

            # Draw bounding box and label
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, f"{labels[label]}: {confidence:.2f}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Drone Object Recognition', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
video_capture.release()
cv2.destroyAllWindows()
